### Mini RAG 

General imports

In [2]:
import os
import pdb
import dotenv
import csv

In [3]:
from pathlib import Path
from uuid import uuid4

In [4]:
# Langchain controls
from langchain.globals import set_debug

Langchain imports

In [5]:
# llm model
from langchain_openai import ChatOpenAI

# Vector database
from langchain_core.vectorstores import InMemoryVectorStore

# Embeddings
from langchain_openai import OpenAIEmbeddings

# Prompts and messages
from langchain.prompts import (
  PromptTemplate, ChatPromptTemplate
)

# Documents processing
import fitz
from langchain.schema import Document

# Splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parsers and runnables
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# Chains
from langchain.chains.retrieval_qa.base import RetrievalQA

In [6]:
# Load environment variables
dotenv.load_dotenv()

True

In [7]:
# Langchain debug mode
set_debug(False)

In [8]:
# Load LLM model as chat model
model_params = {
  "model_name": "gpt-4o-mini",
  "api_version": "2023-05-15",
  "temperature": 0.05, 
  "max_tokens": 4000,
  "top_p": 0.95,
}
    
llm = ChatOpenAI(
  model=model_params["model_name"],
  api_key=os.environ.get('OPENAI_API_KEY'),
  temperature=model_params["temperature"],
  max_tokens=model_params["max_tokens"],
  top_p=model_params["top_p"]
)

In [9]:
# Load embeddings model
embeddings_model: str="text-embedding-ada-002"
embeddings_service = OpenAIEmbeddings(
  model=embeddings_model,
  api_key=os.environ.get('OPENAI_API_KEY')
)

In [10]:
# Create a vector database
#* There are more robust alternatives to this, such as Pinecone, Chroma
vdb = InMemoryVectorStore(embeddings_service)

#### Document source: CSV file

In [11]:
# Load a .csv file
file_folder = "documents"
file_name = "mi-tax-table.csv"
file_path = os.path.join(file_folder, file_name)

documents: list = []
with open(file_path, 'r', encoding='utf-8') as f:
  reader = csv.reader(f)
  header = next(reader)
  rows = [row for row in reader]
  
# Crear documentos para insertar en la vdb
for row in rows:
  doc = Document(
    id=str(uuid4()),
    page_content=" ".join(row),
    metadata={
      "source": file_name
    }
  )
  documents.append(doc)
  
documents[0]

Document(id='433a43ec-69d3-4baf-91ae-0cd48d48d782', metadata={'source': 'mi-tax-table.csv'}, page_content='ST LOUIS COUNTY TAX 1% 66.600-66.630')

In [ ]:
# Add the documents to the vector database
vdb.add_documents(documents)

In [13]:
# Search the vector database by similarity
query = "¿What is st louis county tax rate?"
results = vdb.similarity_search_with_score(
  query=query,
  k=2
)
for result in results:
  print("Answer:::", result[0].page_content)
  print("Score:::", result[1])
  print("*" * 20)

Answer::: ST LOUIS COUNTY TAX 1% 66.600-66.630
Score::: 0.9024074211674541
********************
Answer::: ST LOUIS COUNTY ADDITIONAL SALES TAX 275/1000% 67.581
Score::: 0.877963188931247
********************


In [ ]:
# qa chain
# Retriever
retriever = vdb.as_retriever(
  search_type="similarity",
  search_kwargs={
    "k": 3
  }
)

# Prompt for the qa chain
prompt_text = """
You are an assistant specialized in answering questions about documents.
Your task is to use the information provided in the context to answer
the question.
Instructions: 
  1. Answer the following question based on the information
  2. Do not include unsolicited information, do not make up data, do not 
  include recommendations outside of the provided context.

Context:
{context}
Question:
{question}
"""

qa_prompt = PromptTemplate.from_template(template=prompt_text)

qa_chain = RetrievalQA.from_chain_type(
  llm = llm,
  retriever=retriever,
  return_source_documents=True,
  chain_type="stuff",
  chain_type_kwargs={"prompt": qa_prompt}
)



In [21]:
# QA test
query = "¿What is county tourism sales tax rate in Missouri?"
answer = qa_chain.invoke(
  {"query": query}
)
print("Query:", answer["query"])
print("Answer:", answer["result"])
print("Source documents:")
for doc in answer["source_documents"]:
  print(doc.page_content)

Query: ¿What is county tourism sales tax rate in Missouri?
Answer: The county tourism sales tax rate in Missouri can be up to 7/8%.
Source documents:
COUNTY TOURISM SALES TAX UP TO 7/8% 67.671-67.685
ST LOUIS COUNTY ZOOGICAL SALES TAX 1/8, 1/4, 3/8, 1/2% (Cannot exceed 1/8% beginning 8/28/17) (Cannot be in excess of 1% combined) 67.547
ST LOUIS COUNTY ADDITIONAL SALES TAX 275/1000% 67.581


In [22]:
# Straightforward LLM call
llm_response = llm.invoke(query)
print(llm_response.content)

In Missouri, the county tourism sales tax rate can vary by county, as each county has the authority to impose its own sales tax for tourism-related purposes. Generally, the rates can range from 1% to 5%, depending on the specific county's regulations and decisions. To find the exact tourism sales tax rate for a particular county in Missouri, it's best to check with the local county government or their official website for the most accurate and up-to-date information.


#### Document source: PDF document

In [ ]:
# Load PDF document
file_folder = "documents"
file_name = "wa-taxes.pdf"
file_path = os.path.join(file_folder, file_name)

with open(file_path, "rb") as f:
  document_bytes = f.read()
  
# Open the PDF document
document_pdf = fitz.open(
  stream=document_bytes,
  filetype="pdf"
)

# Split the PDF document into chunks - to insert into the vdb
splitting_method="recursive"
splitter_params = {
  "chunk_size": 800,
  "chunk_overlap": 50
}
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=splitter_params["chunk_size"],
  chunk_overlap=splitter_params["chunk_overlap"]
)

# Create the langchain list of documents
document_with_metadata: list = []
for page_number in range(document_pdf.page_count):
  page = document_pdf.load_page(page_number)
  page_text = page.get_text("text")
  document_with_metadata.append(
    Document(
      id=str(uuid4()),
      page_content=page_text,
      metadata={
        "titulo": file_name,
        "pagina": page_number 
      }
    )
  )
  
document_splitted = text_splitter.split_documents(
  documents=document_with_metadata
)

# Ingest the documents into the vector database
vdb.add_documents(documents=document_splitted)

In [18]:
# semanctic search
query = "¿What are Tax rates for Zoo & Aquarium in WA?"
results = vdb.similarity_search_with_score(
  query=query,
  k=2
)
for result in results:
  print("Answer:::", result[0].page_content)
  print("Score:::", result[1])
  print("*" * 20)

Answer::: Local Sales & Use Tax Components 
 
33 
 
Component Name: 
 
Zoo & Aquarium 
 
Revised Code of WA (RCW): 82.14.400 
Date Legislation Enacted: 
1999 
Date Last Modified: 
 
2020 
Statutory Expiration Date: 
None 
Imposed by: 
 
 
Voter Approval 
 
Description 
• 
Counties may levy a sales and use tax to fund costs associated with a zoo, aquarium, or wild life preservation, 
and parks 
 
Taxing Authorities  
• 
A county with a national park and a population of more than five hundred thousand and less than 1.5 million 
by joint request with metropolitan park district and a city with a population of more than one hundred fifty 
thousand 
 
Tax Base 
• 
This tax applies to retail sales taxable activity under chapter 82.08 and 82.12 RCW 
 
Tax Rate 
 
• 
Up to 0.001 
 
Fiscal Year Cap 
•
Score::: 0.8863987156271392
********************
Answer::: Local Sales & Use Tax Components 
 
34 
 
Line Code(s) 
 
• 
Sales Tax = 95  
• 
Use Tax = 96 
 
Use of funds 
• 
Costs associated with fi

In [19]:
# qa chain
answer = qa_chain.invoke(
  {"query": query}
)
print("Query:", answer["query"])
print("Answer:", answer["result"])
print("Source documents:")
for doc in answer["source_documents"]:
  print(doc.page_content)

Query: ¿What are Tax rates for Zoo & Aquarium in WA?
Answer: The tax rate for Zoo & Aquarium in Washington is up to 0.001.
Source documents:
Local Sales & Use Tax Components 
 
33 
 
Component Name: 
 
Zoo & Aquarium 
 
Revised Code of WA (RCW): 82.14.400 
Date Legislation Enacted: 
1999 
Date Last Modified: 
 
2020 
Statutory Expiration Date: 
None 
Imposed by: 
 
 
Voter Approval 
 
Description 
• 
Counties may levy a sales and use tax to fund costs associated with a zoo, aquarium, or wild life preservation, 
and parks 
 
Taxing Authorities  
• 
A county with a national park and a population of more than five hundred thousand and less than 1.5 million 
by joint request with metropolitan park district and a city with a population of more than one hundred fifty 
thousand 
 
Tax Base 
• 
This tax applies to retail sales taxable activity under chapter 82.08 and 82.12 RCW 
 
Tax Rate 
 
• 
Up to 0.001 
 
Fiscal Year Cap 
•
Local Sales & Use Tax Components 
 
34 
 
Line Code(s) 
 
• 
Sales

In [20]:
# Straightforward LLM call
llm_response = llm.invoke(query)
print(llm_response.content)

In Washington State, zoos and aquariums are generally subject to the state's sales tax, which is currently 6.5% at the state level. However, local jurisdictions may impose additional sales taxes, which can vary by location. This means that the total sales tax rate for admissions and related sales at zoos and aquariums can be higher depending on the specific city or county.

Additionally, zoos and aquariums may also be subject to other taxes, such as the business and occupation (B&O) tax, which is based on gross receipts.

For the most accurate and up-to-date information, it's best to consult the Washington State Department of Revenue or the specific zoo or aquarium in question, as they can provide details on applicable tax rates and any exemptions that may apply.
